In [1]:
!pip3 install imagecorruptions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.1 MB/s eta 0:00:00


In [2]:
from imagecorruptions import corrupt

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tensorflow import keras
import os
import cv2
import random as rn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from keras.utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.applications import VGG16,VGG19,Xception,InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from keras.regularizers import l2
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten,Dropout,Softmax,GlobalAveragePooling2D,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img
import random
# from imagecorruptions import corrupt
import pickle



In [4]:
def mask_surrounding_area(img):
    # Load the image
    # img = cv2.imread(image_path)
    # if img is None:
    #     print("Error: Image could not be read.")
    #     return

    # Get image dimensions
    height, width = 224,224

    # Calculate dimensions for the central 70%
    new_width = int(width * 0.7)
    new_height = int(height * 0.7)

    # Calculate the top-left corner of the bounding box
    x = (width - new_width) // 2
    y = (height - new_height) // 2

    # Create a mask of zeros (black) with the same dimensions as the image
    mask = np.zeros_like(img)

    # Set the central 70% area of the mask to white (255)
    mask[y:y+new_height, x:x+new_width] = 255

    # Apply the mask using bitwise operations
    masked_image = cv2.bitwise_and(img, mask)
    return  masked_image



In [5]:
def preprocess_contours(binary_array):
    final_array = []
    for img in binary_array:
        integer_image = img.astype(np.uint8)

        scaled_image = integer_image * 255
        # i = detect_longest_connected_components(scaled_image)
        mask = mask_surrounding_area(scaled_image)
        final_array.append(scaled_image)
    return np.array(final_array)


In [6]:
with open("/kaggle/input/belgium-nsct-mci-dataset/train_0_15.pkl", 'rb') as file:
    images = pickle.load(file)
X_test = images["train"]["binary"]
y_test = images["train"]["target"]

with open("/kaggle/input/belgium-nsct-mci-dataset/bel_train_16_31.pkl", 'rb') as file:
    images = pickle.load(file)
X_test2 = images["train"]["binary"]
y_test2= images["train"]["target"]

with open("/kaggle/input/belgium-nsct-mci-dataset/train_32_47.pkl", 'rb') as file:
    images = pickle.load(file)
X_test3 = images["train"]["binary"]
y_test3= images["train"]["target"]
with open("/kaggle/input/belgium-nsct-mci-dataset/belgium_train_48_61.pkl", 'rb') as file:
    images = pickle.load(file)
X_test4 = images["train"]["binary"]
y_test4 = images["train"]["target"]
# X_test4 = preprocess_contours(X_test4)
combined_data = np.concatenate((X_test, X_test2,X_test3,X_test4), axis=0)
lab_data = np.concatenate((y_test, y_test2,y_test3,y_test4), axis=0)
print(combined_data.shape)

(3919, 224, 224, 3)


In [7]:
with open("/kaggle/input/belgium-nsct-mci-dataset/test.pkl", 'rb') as file:
    images = pickle.load(file)
X_test= images["train"]["binary"]
y_test = images["train"]["target"]

# with open("/kaggle/input/china-mci-final-dataset/china_test_mci_m_final.pkl", 'rb') as file:
#     images = pickle.load(file)
# X_test2 = images["train"]["binary"]
# y_test2= images["train"]["target"]
# X_test2 = preprocess_contours(X_test2)
# X_test = np.concatenate((X_test1, X_test2), axis=0)
# y_test = np.concatenate((y_test1, y_test2), axis=0)
print(X_test.shape)

(2520, 224, 224, 3)


In [8]:
print(np.unique(y_test))

[ 0  1  2  3  4  5  6  7  8 10 12 13 14 16 17 18 19 20 21 22 23 24 25 27
 28 29 30 31 32 34 35 37 38 39 40 41 42 43 44 45 46 47 49 51 53 54 55 56
 57 58 59 60 61]


In [9]:
X_train = np.array(combined_data)
y_train = np.array(lab_data)


In [10]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(3919, 224, 224, 3) (3919,) (2520, 224, 224, 3) (2520,)


In [11]:
np.unique(y_test)


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 12, 13, 14, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 34, 35, 37, 38, 39,
       40, 41, 42, 43, 44, 45, 46, 47, 49, 51, 53, 54, 55, 56, 57, 58, 59,
       60, 61], dtype=int32)

In [12]:
class_to_delete = 38

# Find indices of elements to delete
indices_to_delete = np.where(y_test == class_to_delete)[0]

# Delete corresponding elements from X_test and y_test
X_test = np.delete(X_test, indices_to_delete, axis=0)
y_test = np.delete(y_test, indices_to_delete)

print("Updated X_test:")
print(X_test.shape)
print("Updated y_test:")
print(y_test.shape)


Updated X_test:
(2307, 224, 224, 3)
Updated y_test:
(2307,)


In [13]:
class_to_delete = 38

# Find indices of elements to delete
indices_to_delete = np.where(y_train == class_to_delete)[0]

X_train = np.delete(X_train, indices_to_delete, axis=0)
y_train = np.delete(y_train, indices_to_delete)
print(X_train.shape)

(3786, 224, 224, 3)


In [14]:
print("Updated X_test:")
print(X_train.shape)
print("Updated y_test:")
print(y_train.shape)

Updated X_test:
(3786, 224, 224, 3)
Updated y_test:
(3786,)


In [15]:
print(X_test.shape,y_test.shape)
print(np.unique(y_test))

(2307, 224, 224, 3) (2307,)
[ 0  1  2  3  4  5  6  7  8 10 12 13 14 16 17 18 19 20 21 22 23 24 25 27
 28 29 30 31 32 34 35 37 39 40 41 42 43 44 45 46 47 49 51 53 54 55 56 57
 58 59 60 61]


In [16]:
num_classes = 61
new_labels = []
mapping = {}
print(np.unique(y_test))
for label in np.unique(y_train):
    if label < 38:
        new_labels.append(label)
    elif label > 38:
        new_labels.append(label - 1)
    mapping[label] = new_labels[-1]
print(mapping)

# Update labels in y_train using the mapping
y_train = np.array([mapping[label] for label in y_train])
y_test = np.array([mapping[label] for label in y_test])

print("Original labels:")
print(np.unique(y_train))



[ 0  1  2  3  4  5  6  7  8 10 12 13 14 16 17 18 19 20 21 22 23 24 25 27
 28 29 30 31 32 34 35 37 39 40 41 42 43 44 45 46 47 49 51 53 54 55 56 57
 58 59 60 61]
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 58: 57, 59: 58, 60: 59, 61: 60}
Original labels:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60]


In [17]:
# num_classes = 62
# new_labels = []
# mapping = {}

# # for label in np.unique(y_test):
#     if label < 38:
#         new_labels.append(label)
#     elif label > 38:
#         new_labels.append(label - 1)
#     mapping[label] = new_labels[-1]

# # Update labels in y_train using the mapping
# y_train = np.array([mapping[label] for label in y_test])
# # y_test = np.array([mapping[label] for label in y_test])

# print("Original labels:")
# print(np.unique(y_test))



In [18]:
## Augmentation to balance class imbalance

MIN_IMGS_IN_CLASS = 74
def count_images_in_classes(lbls):
    dct = {}
    for i in lbls:
        if i in dct:
            dct[i] += 1
        else:
            dct[i] = 1
    return dct



def augment_imgs(imgs, p):
    """
    Performs a set of augmentations with with a probability p
    """
    from imgaug import augmenters as iaa
    augs =  iaa.SomeOf((2, 4),
          [
              iaa.Crop(px=(0, 4)), # crop images from each side by 0 to 4px (randomly chosen)
              iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}),
              iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),
              iaa.Affine(rotate=(-45, 45)), # rotate by -45 to +45 degrees)
              iaa.Affine(shear=(-10, 10)) # shear by -10 to +10 degrees
          ])
    
    seq = iaa.Sequential([iaa.Sometimes(p, augs)])
    res = seq.augment_images(imgs)
    return res
    
def augmentation(imgs, lbls):
    classes = count_images_in_classes(lbls)
    for i in range(len(classes)): 
    
            if (classes[i] < MIN_IMGS_IN_CLASS):
                # Number of samples to be added
                add_num = MIN_IMGS_IN_CLASS - classes[i]
                imgs_for_augm = []
                lbls_for_augm = []
                for j in range(add_num):
                    im_index = random.choice(np.where(lbls == i)[0])
                    imgs_for_augm.append(imgs[im_index])
                    lbls_for_augm.append(lbls[im_index])
                augmented_class = augment_imgs(imgs_for_augm, 1)
                augmented_class_np = np.array(augmented_class)
                augmented_lbls_np = np.array(lbls_for_augm)
                imgs = np.concatenate((imgs, augmented_class_np), axis=0)
                lbls = np.concatenate((lbls, augmented_lbls_np), axis=0)
    return (imgs, lbls)
   
    
print(count_images_in_classes(y_train))
train_X, train_Y = augmentation(X_train, y_train)

{0: 15, 1: 100, 2: 11, 3: 12, 4: 14, 5: 11, 6: 17, 7: 137, 8: 27, 9: 15, 10: 21, 11: 5, 12: 18, 13: 85, 14: 40, 15: 9, 16: 9, 17: 76, 18: 81, 19: 231, 20: 42, 21: 39, 22: 363, 23: 12, 24: 48, 25: 40, 26: 6, 27: 18, 28: 125, 29: 33, 30: 37, 31: 61, 46: 119, 32: 275, 33: 8, 34: 38, 35: 44, 36: 16, 37: 85, 38: 125, 39: 156, 40: 90, 41: 22, 42: 19, 43: 40, 44: 74, 45: 40, 47: 11, 48: 12, 49: 15, 50: 27, 51: 27, 52: 199, 53: 118, 54: 12, 55: 95, 56: 13, 57: 15, 58: 42, 59: 9, 60: 282}


In [19]:
print(count_images_in_classes(y_test))

{0: 6, 1: 27, 2: 7, 3: 6, 4: 12, 5: 3, 6: 6, 7: 90, 8: 12, 10: 28, 12: 3, 13: 39, 14: 15, 16: 12, 17: 183, 18: 122, 19: 163, 20: 3, 21: 45, 22: 61, 23: 15, 24: 13, 25: 3, 27: 9, 28: 51, 29: 28, 30: 37, 31: 86, 32: 422, 34: 9, 35: 154, 37: 31, 38: 99, 39: 48, 40: 11, 41: 9, 42: 6, 43: 3, 44: 84, 45: 6, 46: 31, 48: 3, 50: 3, 52: 24, 53: 48, 54: 15, 55: 33, 56: 41, 57: 9, 58: 17, 59: 11, 60: 105}


In [20]:
print(train_X[1].shape)
print(train_Y.shape)


(224, 224, 3)
(5924,)


In [21]:
print(X_test.shape,y_test.shape)

(2307, 224, 224, 3) (2307,)


In [22]:
alpha=0.0001
epochs=25
optim = keras.optimizers.Adam(learning_rate=alpha)
monitor = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = 8,restore_best_weights = True, min_delta = 0.01)

In [23]:
# from keras.callbacks import ReduceLROnPlateau
# import tensorflow as tf

# # Define EarlyStopping callback
# early_stopping = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss',  # Monitor validation loss
#     patience=8,          # Number of epochs to wait before stopping
#     restore_best_weights=True,  # Restore weights to the best iteration
#     min_delta=0.01       # Minimum change in the monitored quantity to qualify as improvement
# )

# # Define ReduceLROnPlateau callback
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='val_loss',  # Monitor validation loss
#     factor=0.2,          # Factor by which the learning rate will be reduced
#     patience=5,          # Number of epochs with no improvement after which learning rate will be reduced
#     min_lr=0.0001       # Minimum learning rate allowed after reduction
# )

# # Combine both callbacks
# callbacks = [early_stopping, reduce_lr]

In [24]:
# from tensorflow.keras.applications import MobileNetV2
# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224, 3))

# mobilenet_model = Sequential()
# mobilenet_model.add(base_model)
# #new

# mobilenet_model.add(GlobalAveragePooling2D())
# mobilenet_model.add(Dense(units=512, activation='relu'))
# mobilenet_model.add(BatchNormalization())
# mobilenet_model.add(Dropout(0.4))  # Optional dropout for regularization
# mobilenet_model.add(Dense(units=256, activation='relu'))
# mobilenet_model.add(BatchNormalization())
# mobilenet_model.add(Dropout(0.5))  # Optional dropout for regularization
# mobilenet_model.add(Dense(62, activation='softmax'))
# #new

# model = mobilenet_model
# model.summary()
# model.compile(optimizer = keras.optimizers.legacy.Adam(lr=0.0002), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [25]:
from tensorflow.keras.applications import MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224, 3))

mobilenet_model = Sequential()
mobilenet_model.add(base_model)
#new
mobilenet_model.add(Dropout(0.5))
mobilenet_model.add(Flatten())
# mobilenet_model.add(GlobalAveragePooling2D())
mobilenet_model.add(Dense(units=512, activation='relu'))
mobilenet_model.add(BatchNormalization())
mobilenet_model.add(Dropout(0.5))  # Optional dropout for regularization
mobilenet_model.add(Dense(units=512, activation='relu'))
mobilenet_model.add(BatchNormalization())
mobilenet_model.add(Dense(units=512, activation='relu'))
mobilenet_model.add(BatchNormalization())
mobilenet_model.add(Dropout(0.5))  # Optional dropout for regularization
mobilenet_model.add(Dense(62, activation='softmax'))
#new

model = mobilenet_model
model.summary()
model.compile(optimizer = keras.optimizers.legacy.Adam(lr=0.0002), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

9406464/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 dropout (Dropout)           (None, 7, 7, 1280)        0         
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 512)               32113152  
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                           

/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [26]:


from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.models import save_model
from sklearn.metrics import accuracy_score
import numpy as np
from collections import defaultdict

stratified_splitter = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
val_scores = []
train_scores = []
test_scores = []
misclassified_counts = defaultdict(int)

i = 0
for train_index, test_index in stratified_splitter.split(train_X, train_Y):
    i += 1
    X_train, X_val = train_X[train_index], train_X[test_index]
    y_train, y_val = train_Y[train_index], train_Y[test_index]
    
    # Compile and fit the model
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), steps_per_epoch=(len(X_train) // 16), epochs=25, verbose=1, callbacks=monitor) 
    model.save(f"/kaggle/working/Belgium_mobile_newest_1_NSCT_MCI_m_3_{i}.h5")
    
    # Validation set predictions
    pred_val = model.predict(X_val).argmax(axis=-1)
    val_acc = accuracy_score(y_val, pred_val)
    val_scores.append(val_acc)
    
    # Training set predictions
    pred_train = model.predict(X_train).argmax(axis=-1)
    train_acc = accuracy_score(y_train, pred_train)
    train_scores.append(train_acc)
    
    # Test set predictions
    pred_test = model.predict(X_test).argmax(axis=-1)
    test_acc = accuracy_score(y_test, pred_test)
    print(test_acc)
    test_scores.append(test_acc)
    
    # Identify misclassified classes in validation set
    misclassified = np.where(pred_val != y_val)[0]
    for idx in misclassified:
        true_label = y_val[idx]
        predicted_label = pred_val[idx]
        misclassified_counts[(true_label, predicted_label)] += 1

# Print overall misclassified classes
# print("Overall misclassified classes:")
# for (true_label, predicted_label), count in misclassified_counts.items():
#     print(f"True label: {true_label}, Predicted label: {predicted_label}, Count: {count}")

# Print average scores
print(f"Average validation accuracy: {np.mean(val_scores)}")
print(f"Average training accuracy: {np.mean(train_scores)}")
print(f"Average test accuracy: {np.mean(test_scores)}")


Epoch 1/25


2024-06-06 20:49:06.414900: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


296/296 [==============================] - 40s 100ms/step - loss: 2.9627 - accuracy: 0.3443 - val_loss: 2.3691 - val_accuracy: 0.4169
Epoch 2/25
296/296 [==============================] - 28s 95ms/step - loss: 1.1315 - accuracy: 0.6911 - val_loss: 1.2208 - val_accuracy: 0.6439
Epoch 3/25
296/296 [==============================] - 28s 95ms/step - loss: 0.5368 - accuracy: 0.8490 - val_loss: 0.5301 - val_accuracy: 0.8464
Epoch 4/25
296/296 [==============================] - 28s 94ms/step - loss: 0.2706 - accuracy: 0.9254 - val_loss: 0.2953 - val_accuracy: 0.9190
Epoch 5/25
296/296 [==============================] - 28s 94ms/step - loss: 0.1784 - accuracy: 0.9545 - val_loss: 0.1395 - val_accuracy: 0.9536
Epoch 6/25
296/296 [==============================] - 28s 93ms/step - loss: 0.1429 - accuracy: 0.9620 - val_loss: 0.1054 - val_accuracy: 0.9688
Epoch 7/25
296/296 [==============================] - 28s 94ms/step - loss: 0.0955 - accuracy: 0.9735 - val_loss: 0.0466 - val_accuracy: 0.9882
Ep

In [27]:

print("Average for train set:",np.mean(train_scores))
print("Std Deviation for train set:",np.std(train_scores))
print("Average for val set:",np.mean(val_scores))
print("Std Deviation for val set:",np.std(val_scores))
print("Average for test set:",np.mean(test_scores))
print("Std Deviation for test set:",np.std(test_scores))

Average for train set: 0.9982696771470774
Std Deviation for train set: 0.0011816838995568609
Average for val set: 0.9940928270042194
Std Deviation for val set: 0.003958156759344651
Average for test set: 0.9641959254443
Std Deviation for test set: 0.006485168343048114


In [28]:
# from tensorflow.keras.applications import MobileNetV2
# mb=MobileNetV2(include_top=False,input_shape=(224,224,3),weights='imagenet')
# model = Sequential()
# model.add(mb)
# model.add(Dropout(0.5))
# model.add(Flatten())
# model.add(Dense(units = 512, activation = 'relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(Dense(units = 512, activation = 'relu'))
# model.add(BatchNormalization())
# model.add(Dense(units =512 , activation = 'relu'))
# model.add(BatchNormalization())
# model.add(Dense(units = 62 , activation = 'softmax'))

# model.summary()
# model.compile(optimizer = keras.optimizers.legacy.Adam(lr=0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [29]:
# from sklearn.model_selection import StratifiedShuffleSplit
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
# from tensorflow.keras.models import save_model
# from sklearn.metrics import accuracy_score
# import numpy as np
# from collections import defaultdict

# stratified_splitter = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
# val_scores = []
# train_scores = []
# test_scores = []
# misclassified_counts = defaultdict(int)

# i = 0
# for train_index, test_index in stratified_splitter.split(train_X, train_Y):
#     i += 1
#     X_train, X_val = train_X[train_index], train_X[test_index]
#     y_train, y_val = train_Y[train_index], train_Y[test_index]
    
#     # Compile and fit the model
#     history = model.fit(X_train, y_train, validation_data=(X_val, y_val), steps_per_epoch=(len(X_train) // 16), epochs=25, verbose=1, callbacks=monitor) 
#     model.save(f"/kaggle/working/Belgium_mobile_NSCT_MCI_m_3_{i}.h5")
    
#     # Validation set predictions
#     pred_val = model.predict(X_val).argmax(axis=-1)
#     val_acc = accuracy_score(y_val, pred_val)
#     val_scores.append(val_acc)
    
#     # Training set predictions
#     pred_train = model.predict(X_train).argmax(axis=-1)
#     train_acc = accuracy_score(y_train, pred_train)
#     train_scores.append(train_acc)
    
#     # Test set predictions
#     pred_test = model.predict(X_test).argmax(axis=-1)
#     test_acc = accuracy_score(y_test, pred_test)
#     print(test_acc)
#     test_scores.append(test_acc)
    
#     # Identify misclassified classes in validation set
#     misclassified = np.where(pred_val != y_val)[0]
#     for idx in misclassified:
#         true_label = y_val[idx]
#         predicted_label = pred_val[idx]
#         misclassified_counts[(true_label, predicted_label)] += 1

# # Print overall misclassified classes
# # print("Overall misclassified classes:")
# # for (true_label, predicted_label), count in misclassified_counts.items():
# #     print(f"True label: {true_label}, Predicted label: {predicted_label}, Count: {count}")

# # Print average scores
# print(f"Average validation accuracy: {np.mean(val_scores)}")
# print(f"Average training accuracy: {np.mean(train_scores)}")
# print(f"Average test accuracy: {np.mean(test_scores)}")


In [30]:
# import gc
# from sklearn.model_selection import StratifiedShuffleSplit
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
# from tensorflow.keras.models import save_model
# from tensorflow.keras import backend as K
# import numpy as np

# # Assuming train_X and train_Y are already defined
# # Assuming X_test and y_test are already defined

# stratified_splitter = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
# val_scores = []
# train_scores = []
# test_scores = []

# for i, (train_index, test_index) in enumerate(stratified_splitter.split(train_X, train_Y), 1):
#     X_train, X_val = train_X[train_index], train_X[test_index]
#     y_train, y_val = train_Y[train_index], train_Y[test_index]

# #     model.compile(optimizer = optim, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
#     history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
#                         steps_per_epoch=len(X_train) // 16, epochs=25, verbose=1, callbacks=[monitor])

#     # Save the model
#     model.save(f"/kaggle/working/Belgium_MB_WO_3_{i}.h5")

# #     # Evaluate the model
# #     pred_val = model.predict(X_val).argmax(axis=-1)
# #     val_acc = accuracy_score(y_val, pred_val)
# #     val_scores.append(val_acc)

# #     pred_train = model.predict(X_train).argmax(axis=-1)
# #     train_acc = accuracy_score(y_train, pred_train)
# #     train_scores.append(train_acc)

# #     pred_test = model.predict(X_test).argmax(axis=-1)
# #     test_acc = accuracy_score(y_test, pred_test)
# #     print(test_acc)
# #     test_scores.append(test_acc)

#     # Clear session to free up memory
#     K.clear_session()
#     gc.collect()

# print(f"Validation scores: {val_scores}")
# print(f"Training scores: {train_scores}")
# print(f"Test scores: {test_scores}")

In [31]:
# from keras.models import load_model
# res_1 = load_model("/kaggle/working/Belgium_mobile_NSCT_MCI_m_3_1.h5")
# res_2 = load_model("/kaggle/input/chinese-mci-models/Chinese_RES_MCI_2.h5")
# res_3 = load_model("/kaggle/input/chinese-mci-models/Chinese_RES_MCI_3.h5")
# res_4 = load_model("/kaggle/input/chinese-mci-models/Chinese_RES_MCI_4.h5")
# res_5 = load_model("/kaggle/input/chinese-mci-models/Chinese_RES_MCI_5.h5")

In [32]:
# test_scores = []
# pred = res_1.predict(X_train,verbose=0)
# pred = pred.argmax(axis=-1)
# test_scores.append(accuracy_score(y_train, pred))

# pred = res_2.predict(X_train,verbose=0)
# pred = pred.argmax(axis=-1)
# test_scores.append(accuracy_score(y_train, pred))

# pred = res_3.predict(X_train,verbose=0)
# pred = pred.argmax(axis=-1)
# test_scores.append(accuracy_score(y_train, pred))

# pred = res_4.predict(X_train,verbose=0)
# pred = pred.argmax(axis=-1)
# test_scores.append(accuracy_score(y_train, pred))

# pred = res_5.predict(X_train,verbose=0)
# pred = pred.argmax(axis=-1)
# test_scores.append(accuracy_score(y_train, pred))

# print(np.mean(test_scores))

In [33]:
# import numpy as np
# from sklearn.metrics import confusion_matrix

# cm = confusion_matrix(y_test, pred)

# # Display the confusion matrix
# print("Confusion Matrix:")
# print(cm)

In [34]:
# from keras.models import load_model
# res_1 = load_model("/kaggle/working/Belgium_mobile_NSCT_MCI_m_3_1.h5")
# res_2 = load_model("/kaggle/input/chinese-mci-models/Chinese_RES_MCI_2.h5")
# res_3 = load_model("/kaggle/input/chinese-mci-models/Chinese_RES_MCI_3.h5")
# res_4 = load_model("/kaggle/input/chinese-mci-models/Chinese_RES_MCI_4.h5")
# res_5 = load_model("/kaggle/input/chinese-mci-models/Chinese_RES_MCI_5.h5")

In [35]:
# test_scores = []
# # train_scores =
# pred = res_1.predict(X_test,verbose=0)
# pred = pred.argmax(axis=-1)
# test_scores.append(accuracy_score(y_test, pred))

# pred = res_2.predict(X_test,verbose=0)
# pred = pred.argmax(axis=-1)
# test_scores.append(accuracy_score(y_test, pred))

# pred = res_3.predict(X_test,verbose=0)
# pred = pred.argmax(axis=-1)
# test_scores.append(accuracy_score(y_test, pred))

# pred = res_4.predict(X_test,verbose=0)
# pred = pred.argmax(axis=-1)
# test_scores.append(accuracy_score(y_test, pred))

# pred = res_5.predict(X_test,verbose=0)
# pred = pred.argmax(axis=-1)
# test_scores.append(accuracy_score(y_test, pred))

# print(np.mean(test_scores))

In [36]:
# #### Without

# from tensorflow.keras.applications import MobileNetV2
# mb=MobileNetV2(include_top=False,input_shape=(96,96,3),weights='imagenet')
# mb_model = Sequential()
# mb_model.add(mb)
# mb_model.add(Dropout(0.5))
# mb_model.add(Flatten())
# mb_model.add(Dense(units = 512, activation = 'relu'))
# mb_model.add(BatchNormalization())
# mb_model.add(Dense(units = 512, activation = 'relu'))
# mb_model.add(BatchNormalization())
# mb_model.add(Dense(units =512 , activation = 'relu'))
# mb_model.add(BatchNormalization())
# mb_model.add(Dense(units = 58 , activation = 'softmax'))

# mb_model.summary()

In [37]:
# from keras.applications import ResNet50
# from keras.regularizers import l2

# resnet = ResNet50(weights='imagenet',pooling='avg', include_top=False, input_shape=(224,224,3))
# model = Sequential()
# model.add(resnet)
# model.add(Flatten())
# model.add(Dense(512,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(512,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(512,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(512,activation='relu'))
# model.add( Dense(57, activation= 'softmax'))
# model.summary()
# model.compile(optimizer = keras.optimizers.legacy.Adam(lr=0.0002), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])


In [38]:
# from keras.layers import Dropout
# from keras.applications import ResNet50
# # from keras.regularizers import l2
# # Define the model
# resnet = ResNet50(weights='imagenet',pooling='avg', include_top=False, input_shape=(224,224,3))
# res_model = Sequential()
# res_model.add(resnet)
# res_model.add(Flatten())
# res_model.add(Dense(units=512, activation='relu', kernel_regularizer=l2(0.001)))
# res_model.add(BatchNormalization())
# res_model.add(Dropout(0.5))  # Add dropout layer
# res_model.add(Dense(units=512, activation='relu', kernel_regularizer=l2(0.001)))
# res_model.add(BatchNormalization())
# res_model.add(Dropout(0.5))  # Add dropout layer
# res_model.add(Dense(units=512, activation='relu', kernel_regularizer=l2(0.001)))
# res_model.add(BatchNormalization())
# res_model.add(Dropout(0.5))  # Add dropout layer
# res_model.add(Dense(units=512, activation='relu', kernel_regularizer=l2(0.001)))
# res_model.add(BatchNormalization())
# res_model.add(Dropout(0.5))  # Add dropout layer
# res_model.add(Dense(units=57, activation='softmax'))

# # Compile the model
# res_model.summary()

In [39]:
# from sklearn.model_selection import StratifiedShuffleSplit

# stratified_splitter = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
# i = 0
# val = []
# train = []
# test = []
# for train_index, test_index in stratified_splitter.split(train_X, train_Y ):
#     i = i +1
#     X_train, X_val = train_X[train_index], train_X[test_index]
#     y_train, y_val =  train_Y[train_index], train_Y[test_index]

#     alpha=0.0001
#     optim = keras.optimizers.legacy.Adam(learning_rate=alpha)
#     monitor = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = 8,restore_best_weights = True, min_delta = 0.01)


#     res_model.compile(optimizer = optim, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

#     history = res_model.fit(X_train,y_train, validation_data=(X_val,y_val),steps_per_epoch=(len(X_train) // 32),epochs = 25,verbose=1,callbacks=[monitor]) 
#     res_model.save(f"/kaggle/working/Chinese_RES_NSCT_MCI_mi_{i}.h5")
#     pred = res_model.predict(X_val)
#     pred = pred.argmax(axis=-1)
#     val.append(accuracy_score(y_val, pred))
# #     print('ResNet Validation Data accuracy: ',accuracy_score(y_val, pred))
#     pred = res_model.predict(X_train)
#     pred = pred.argmax(axis=-1)
#     train.append(accuracy_score(y_train,pred))
#     pred = res_model.predict(X_test)
#     pred = pred.argmax(axis=-1)
#     test.append(accuracy_score(y_test, pred))
    
# avg_val = np.mean(val)
# avg_train = np.mean(train)
# avg_test= np.mean(test)
# std_val = np.std(val)
# std_train = np.std(train)
# std_test = np.std(test)
# # std_test = statistics.stdev(test)
# print('Average train accuracy: ', avg_train,'std dev : ', std_train)
# print('Average validation accuracy ', avg_val, 'std dev : ',std_val)
# print('Average test accuracy: ', avg_test,'std dev : ', std_test)
# print(train)
# print(val)
# print(test)

In [40]:
# from sklearn.model_selection import StratifiedShuffleSplit
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
# from tensorflow.keras.models import save_model

# stratified_splitter = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
# val_scores = []
# train_scores = []
# test_scores = []
# i = 0
# for train_index, test_index in stratified_splitter.split(train_X,train_Y):
#     i=i+1
#     X_train, X_val = train_X[train_index], train_X[test_index]
#     y_train, y_val = train_Y[train_index], train_Y[test_index]
    
# #     model.compile(optimizer = optim, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
#     history = model.fit(X_train,y_train, validation_data=(X_val,y_val),steps_per_epoch=(len(X_train) // 16),epochs = 25,verbose=1,callbacks=monitor) 
#     model.save(f"/kaggle/working/Belgium_mobile_NSCT_MCI_m_3_{i}.h5")
#     pred = model.predict(X_val)
#     pred = pred.argmax(axis=-1)
#     val_acc =accuracy_score(y_val, pred)
#     val_scores.append(val_acc)
#     pred = model.predict(X_train)
#     pred = pred.argmax(axis=-1)
#     train_acc =accuracy_score(y_train, pred)
#     train_scores.append(train_acc)
#     pred = model.predict(X_test)
#     pred = pred.argmax(axis=-1)
#     test_acc =accuracy_score(y_test, pred)
#     print(test_acc)
#     test_scores.append(test_acc)

#     misclassified = np.where(pred_val != y_val)[0]
#     for idx in misclassified:
#         true_label = y_val[idx]
#         predicted_label = pred_val[idx]
#         misclassified_counts[(true_label, predicted_label)] += 1

# # Print overall misclassified classes
# print("Overall misclassified classes:")
# for (true_label, predicted_label), count in misclassified_counts.items():
#     print(f"True label: {true_label}, Predicted label: {predicted_label}, Count: {count}")

# from IPython.display import FileLink
# FileLink(r'/kaggle/working/Chinese_RES_NSCT_MCI_m_1_1.h5')


In [41]:
# ## Withoutt
# print("Average for train set:",np.mean(train_scores))
# print("Std Deviation for train set:",np.std(train_scores))
# print("Average for val set:",np.mean(val_scores))
# print("Std Deviation for val set:",np.std(val_scores))
# print("Average for test set:",np.mean(test_scores))
# print("Std Deviation for test set:",np.std(test_scores))

In [42]:
# import os

# def calculate_average_images_per_class(base_dir):
#     """
#     Calculate the average number of images per class in the dataset.
    
#     Parameters:
#     base_dir (str): The base directory containing class subdirectories.
    
#     Returns:
#     float: The average number of images per class.
#     """
#     # Dictionary to store the count of images for each class
#     class_image_count = {}

#     # Iterate over each class directory
#     for class_name in os.listdir(base_dir):
#         class_path = os.path.join(base_dir, class_name)
        
#         if os.path.isdir(class_path):
#             # Count the number of images in the current class directory
#             num_images = len([img for img in os.listdir(class_path) if img.endswith('.ppm')])
#             class_image_count[class_name] = num_images

#     # Calculate the average number of images per class
#     average_images_per_class = sum(class_image_count.values()) / len(class_image_count)
    
#     return average_images_per_class

# # Path to the directory containing your dataset
# dataset_dir = '/kaggle/input/belgium-ts/BelgiumTSC_Training/Training'

# # Calculate and print the average number of images per class
# average_images = calculate_average_images_per_class(dataset_dir)
# print("Average number of images per class:", average_images)


In [43]:
# print(test_scores)

In [44]:
# def show_images_of_all_classes(train_images,training_labels):
#   num_cols =  8
#   classes = len(set(y_train))
#   if classes % num_cols == 0:
#       num_rows =  classes / num_cols
#   else:
#       num_rows = int(classes / num_cols) + 1
#   plt.figure(figsize=(15,15))
#   i = 1
#   for class_number in set(y_train):
    
#     indices = np.where(y_train == class_number)
#     plt.subplot(num_rows,num_cols, i)
#     i += 1
#     plt.axis('off')
#     plt.imshow(X_train[indices[0][0]])
#     plt.title('Class :{0} [{1}] '.format(class_number,len(indices[0])))   #np.count_nonzero(labels == labels[index])))
#   plt.show()

# # show_images_of_all_classes(train_X,train_Y)

In [45]:
# def add_single_noise(noise,severity,data_directory):
#   images = []
#   labels = []
  
#   directories = [dir for dir in os.listdir(data_directory)
#                  if os.path.isdir(os.path.join(data_directory,dir))] 
#   for dir in directories:
#     current_directory = os.path.join(data_directory,dir)
#     file_names = [os.path.join(current_directory,file) 
#                   for file in os.listdir(current_directory)
#                   if file.endswith('.png')
#                  ]
    
#     for image in file_names:
#       img = cv2.imread(image)
#       image_fromarray = Image.fromarray(img, 'RGB')
#       resize_image = image_fromarray.resize((96, 96))
#       resize_image = np.array(resize_image)
#       corrupted_image = corrupt(resize_image, corruption_name=noise, severity=severity)
#       images.append(corrupted_image)
#       labels.append(int(dir))
# #   print('Successfully Loadded  {0} images!'.format(len(images)))
#   return np.array(images)/255,np.array(labels)

In [46]:
# TEST_DATA_PATH = '/kaggle/input/chinese-mci-model'

In [47]:
# for i in range(1,6):
#             test_scores = []
#             X_test,y_test = add_single_noise('impulse_noise',i,TEST_DATA_PATH)
#             pred = res_1.predict(X_test,verbose=0)
#             pred = pred.argmax(axis=-1)
#             test_scores.append(accuracy_score(y_test, pred))
#             pred = res_2.predict(X_test,verbose=0)
#             pred = pred.argmax(axis=-1)
#             test_scores.append(accuracy_score(y_test, pred))
#             pred = res_3.predict(X_test,verbose=0)
#             pred = pred.argmax(axis=-1)
#             test_scores.append(accuracy_score(y_test, pred))
#             pred = res_4.predict(X_test,verbose=0)
#             pred = pred.argmax(axis=-1)
#             test_scores.append(accuracy_score(y_test, pred))
#             pred = res_5.predict(X_test,verbose=0)
#             pred = pred.argmax(axis=-1)
#             test_scores.append(accuracy_score(y_test, pred))
#             print(np.mean(test_scores))
#             print(np.std(test_scores))
# #             print(test_scores)

In [48]:
# import re

# def update_labels(labels):
#     updated_labels = []
#     for label in labels:
#         # Use regular expression to extract the number before the first underscore
#         match = re.match(r'^(\d+)_', label)
#         if match:
#             # Extract the matched number and convert it to an integer
#             updated_label = int(match.group(1))
#             updated_labels.append(updated_label)
#         else:
#             # If the label doesn't match the specified format, keep it unchanged
#             updated_labels.append(label)
#     return updated_labels




In [49]:
# def mask_surrounding_area(img):
#     # Load the image
#     # img = cv2.imread(image_path)
#     # if img is None:
#     #     print("Error: Image could not be read.")
#     #     return

#     # Get image dimensions
#     height, width = 224,224

#     # Calculate dimensions for the central 70%
#     new_width = int(width * 0.7)
#     new_height = int(height * 0.7)

#     # Calculate the top-left corner of the bounding box
#     x = (width - new_width) // 2
#     y = (height - new_height) // 2

#     # Create a mask of zeros (black) with the same dimensions as the image
#     mask = np.zeros_like(img)

#     # Set the central 70% area of the mask to white (255)
#     mask[y:y+new_height, x:x+new_width] = 255

#     # Apply the mask using bitwise operations
#     masked_image = cv2.bitwise_and(img, mask)
#     return  masked_image



In [50]:
# def preprocess_contours(binary_array):
#     final_array = []
#     for img in binary_array:
#         integer_image = img.astype(np.uint8)

#         scaled_image = integer_image * 255
#         # i = detect_longest_connected_components(scaled_image)
#         mask = mask_surrounding_area(scaled_image)
#         final_array.append(scaled_image)
#     return np.array(final_array)

In [51]:
# import pickle
# for i in range(1,6):
#     with open(f'/kaggle/input/chinesemci-2/shot_{i}.pkl',mode='rb') as f:
#         train = pickle.load(f)
# #         print(train)
#     if True:
#                 test_scores = []
#                 X_test = train["train"]["binary"]
#                 y_test = train["train"]["target"]
#                 output_images = []
#                 for img in X_test:
#                     # Convert image to uint8
#                     img_uint8 = img.astype(np.uint8)
#                     # Apply the function to each image
#                     output_img = detect_longest_connected_components(img_uint8)
#                     output_images.append(output_img)

#                 output_images = np.array(output_images)
# #                 print(output_images.shape)
#                 masked_images = []
#                 for img in output_images:
#                     # Apply the function to each image
#                     masked_img = mask_surrounding_area(img)
#                     masked_images.append(masked_img)

#                 masked_images = np.array(masked_images)
#                 X_test = np.repeat(masked_images[..., np.newaxis], 3, axis=-1)
# #                 print(y_test)
# #                 y_test = update_labels(y_test)
#                 pred = res_1.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_2.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_3.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_4.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_5.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 print(np.mean(test_scores))
#                 print(np.std(test_scores))

In [52]:
# import pickle
# for i in range(1,6):
#     with open(f'/kaggle/input/chinese-noise-corf/Chinese_DB_{i}.pkl',mode='rb') as f:
#         train = pickle.load(f)
# #         print(train)
#     if True:
#                 test_scores = []
#                 X_test = train["test"]["data"]
#                 y_test = train["test"]["target"]
#                 pred = res_1.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_2.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_3.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_4.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_5.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 print(np.mean(test_scores))
#                 print(np.std(test_scores))

In [53]:
# import pickle
# for i in range(1,6):
#     with open(f'/kaggle/input/chinese-noise-corf/Chinese_MB_{i}.pkl',mode='rb') as f:
#         train = pickle.load(f)
# #         print(train)
#     if True:
#                 test_scores = []
#                 X_test = train["test"]["data"]
#                 y_test = train["test"]["target"]
#                 pred = res_1.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_2.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_3.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_4.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_5.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 print(np.mean(test_scores))
#                 print(np.std(test_scores))

In [54]:
# import pickle
# for i in range(1,6):
#     with open(f'/kaggle/input/chinese-noise-corf/Chinese_shot_{i}.pkl',mode='rb') as f:
#         train = pickle.load(f)
# #         print(train)
#     if True:
#                 test_scores = []
#                 X_test = train["test"]["data"]
#                 y_test = train["test"]["target"]
#                 pred = res_1.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_2.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_3.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_4.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 pred = res_5.predict(X_test,verbose=0)
#                 pred = pred.argmax(axis=-1)
#                 test_scores.append(accuracy_score(y_test, pred))
#                 print(np.mean(test_scores))
#                 print(np.std(test_scores))

In [55]:
# os.chdir("/kaggle/working")

In [56]:
# from IPython.display import FileLink
# FileLink("Chinese_MB_CORF_1.h5")

In [57]:
# def add_noise(noise, severity, data_directory):
#     images = []
#     labels = []

#     directories = [dir for dir in os.listdir(data_directory)
#                    if os.path.isdir(os.path.join(data_directory, dir))]

#     for dir in directories:
#         current_directory = os.path.join(data_directory, dir)
#         file_names = [os.path.join(current_directory, file)
#                       for file in os.listdir(current_directory)
#                       if file.endswith('.png')]

#         for image in file_names:
#             img = cv2.imread(image)

#             # Step 1: Resize the image to 96x96
#             image_fromarray = Image.fromarray(img, 'RGB')
#             resize_image = image_fromarray.resize((96, 96))
#             resize_image = np.array(resize_image)
            
#             # Step 2: Apply corruption
#             corrupted_image = corrupt(resize_image, corruption_name=noise, severity=severity)
            
#             # Step 3: Resize the corrupted image to 224x224
#             corrupted_image_fromarray = Image.fromarray(corrupted_image, 'RGB')
#             final_resized_image = corrupted_image_fromarray.resize((224, 224))
#             final_resized_image = np.array(final_resized_image)
            
#             images.append(final_resized_image)
#             labels.append(int(dir))

#     # print('Successfully Loaded {0} images!'.format(len(images)))
#     return np.array(images) / 255, np.array(labels)


# TEST_DATA_PATH = "/kaggle/input/chinese-test/test/test"

In [58]:
# for noise in noises :
#     print(noise)
#     for i in range(1,6):
#             X_test,y_test = add_noise(noise,i,TEST_DATA_PATH)
#             class_to_delete = 54

#                 # Find indices of elements to delete
#             indices_to_delete = np.where(y_test == class_to_delete)[0]

#                 # Delete corresponding elements from X_test and y_test
#             X_test = np.delete(X_test, indices_to_delete, axis=0)
#             y_test = np.delete(y_test, indices_to_delete)
#             num_classes = 57
#             new_labels = []
#             mapping = {}
# #                 print(y_test.shape)
#             for label in np.unique(y_test):
#                     if label < 54:
#                         new_labels.append(label)
#                     elif label > 54:
#                         new_labels.append(label - 1)
#                     mapping[label] = new_labels[-1]

#                 # Update labels in y_train using the mapping
#             y_test = np.array([mapping[label] for label in y_test])
#             test_scores = []


# #             X_test,y_test = add_noise(j,i,data_directory )
# #             print(i)
#             pred = res_1.predict(X_test,verbose=0)
#             pred = pred.argmax(axis=-1)
#             test_scores.append(accuracy_score(y_test, pred))
#             pred = res_2.predict(X_test,verbose=0)
#             pred = pred.argmax(axis=-1)
#             test_scores.append(accuracy_score(y_test, pred))
# #             print(test_scores)
#             pred = res_3.predict(X_test,verbose=0)
#             pred = pred.argmax(axis=-1)
#             test_scores.append(accuracy_score(y_test, pred))
# #             print(test_scores)
#             pred = res_4.predict(X_test,verbose=0)
#             pred = pred.argmax(axis=-1)
#             test_scores.append(accuracy_score(y_test, pred))
# #             print(test_scores)
#             pred = res_5.predict(X_test,verbose=0)
#             pred = pred.argmax(axis=-1)
#             test_scores.append(accuracy_score(y_test, pred))
# #             print(test_scores)
#             print(np.mean(test_scores))
#             print(np.std(test_scores))
#             del test_scores
            
